In [1]:
import numpy as np
import pandas as pd
import utm
import math
import pickle
import os
import sys
sys.path.append('./src')
import time
import datetime
from collections import defaultdict
import matplotlib
import matplotlib.pyplot as plt
from shapely.geometry import Point, LineString
import copy

import display_gm
import display_osm
import plot
import tools
import pygrid
import hmm, uni_hmm, split
import celltower as ctcrawler
from data_cleaning import noise, load, counter

# Load Data

In [267]:
load = reload(load)
dataset = 'jiading'
datatype = '2g'
data_file = './data/%s_%s/data_%s.csv' % (dataset, datatype, datatype)
gongcan_file = './data/%s_%s/gongcan_%s.csv' % (dataset, datatype, datatype)
disp_path = './display/%s_%s/' % (dataset, datatype)
data_path = './data/%s_%s/' % (dataset, datatype)
map_file = './data/%s_map/%s_EdgeGeometry.txt' % (dataset, dataset)

In [434]:
db, db_gps, towers = load.load_data(data_file, gongcan_file, merge_tower=False, neighbor=True, with_rssi=False, radio_angle=False, context=False)
print 'len(db):', len(db), 'len(towers):', len(towers)

Unique Cell Tower:  81
Duplicated cell towers: [[66, 67, 68], [35, 36], [53, 54], [69, 70, 71], [40, 41], [56, 57], [61, 62, 63]]
Totally duplicate: 86
len(db): 72 len(towers): 91


In [435]:
db_gps[0][:5]

[(31.29184524, 121.21368387, 62, 70, 0.0, 1507627341L),
 (31.28926598, 121.21361016, 62, 70, 0.68, 1507627346L),
 (31.28922812, 121.21357512, 63, 70, 0.0, 1507627351L),
 (31.28926254, 121.21358264, 63, 70, 0.0, 1507627356L),
 (31.28928476, 121.21360502, 63, 70, 0.0, 1507627359L)]

# Data Cleaning

In [436]:
noise = reload(noise)
max_dist, min_dist, min_len, max_again = noise.get_config(dataset, datatype)
dbs = noise.clean_db(db, max_dist, min_dist, min_len, max_again, debug=True)
dbs_gps = noise.clean_db_gps(db_gps, max_dist, min_dist, min_len, max_again, debug=False)
len(dbs), len(dbs_gps)

Traj ID=0
0->1:286m	
discard[0:0]=0m	keep[1:25]as id=0	
Traj ID=1
7->8:510m	
discard[0:7]=0m	keep[8:158]as id=1	
Traj ID=2
1->2:330m	
discard[0:1]=0m	keep[2:78]as id=2	
Traj ID=3

keep[0:34]as id=3	
Traj ID=4
0->1:394m	
discard[0:0]=0m	keep[1:45]as id=4	
Traj ID=5
1->2:118m	
discard[0:1]=0m	keep[2:50]as id=5	
Traj ID=6
1->2:522m	112->113:752m	133->134:116m	
discard[0:1]=0m	trimed[0,89] keep[2:23]as id=6	discard[113:133]=33m	discard[134:135]=0m	
Traj ID=7
9->10:101m	
discard[0:9]=0m	keep[10:113]as id=7	
Traj ID=8

keep[0:374]as id=8	
Traj ID=9
1->2:947m	
discard[0:1]=0m	keep[2:342]as id=9	
Traj ID=10

keep[0:421]as id=10	
Traj ID=11

keep[0:296]as id=11	
Traj ID=12

keep[0:423]as id=12	
Traj ID=13
35->36:838m	
discard[0:35]=0m	keep[36:1016]as id=13	
Traj ID=14

keep[0:827]as id=14	
Traj ID=15
42->43:667m	
keep[0:43]as id=15	keep[43:115]as id=16	
Traj ID=16
2->3:482m	345->346:714m	
discard[0:2]=0m	keep[3:346]as id=17	keep[346:854]as id=18	
Traj ID=17

trimed[1,12] keep[1:234]as id=19	
Tr

(74, 74)

In [419]:
db_gps[0][:5]

[(31.29184524, 121.21368387, 62, 0.0, 1507627341L),
 (31.28926598, 121.21361016, 62, 0.68, 1507627346L),
 (31.28922812, 121.21357512, 63, 0.0, 1507627351L),
 (31.28926254, 121.21358264, 63, 0.0, 1507627356L),
 (31.28928476, 121.21360502, 63, 0.0, 1507627359L)]

# Ground Truth

In [437]:
load = reload(load)
match_res, time_set = load.load_matching(data_path + 'matching_out', len(dbs), 50)
dbs = noise.reclean(dbs, time_set, debug=True)
dbs_gps = noise.reclean(dbs_gps, time_set)

traj id= 1 150 140
traj id= 13 980 976
traj id= 22 190 167
traj id= 23 216 198
traj id= 25 427 420
traj id= 26 185 149
traj id= 41 181 173
traj id= 44 136 131
traj id= 50 219 218
traj id= 51 322 316
traj id= 63 34 32
traj id= 67 89 87
traj id= 72 50 44


# Split Data

In [438]:
split = reload(split)
# train:(origin idx, origin feature)
# train, test = hmm.k_split(dbs, k=4)
train, test, gps3 = split.k_g_split(dbs, k=4, gpsize=0.0)
# train, test = hmm.hand_split(dbs, k=2)
# len(gps3)

In [439]:
train_set = set()
for tr_id, traj in train.iteritems():
    for point in traj:
        train_set.add(point[1][2:-2])
test_set = set()
cnt = 0
for tr_id, traj in test.iteritems():
    for point in traj:
        obsv = point[1][2:-2]
        if obsv not in train_set:
            cnt += 1
        test_set.add(point[1][2:-2])
test_set-train_set, cnt

({(7, 32), (9, -1), (61, 62), (68, -1), (68, 67)}, 5)

In [303]:
bounding_box = (328000, 331000, 3462000, 3465000)
side = 30
grid = pygrid.Grid(side, bounding_box)

In [304]:
tools = reload(tools)

# Slice Roadmap

In [336]:
roadmap = load.load_map(map_file)
slice_map = tools.make_map(grid, roadmap, 150, 1)

In [337]:
for rid in slice_map.iterkeys():
    if len(slice_map[rid]) == 0:
        print rid

# Layer 1

In [343]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.naive_bayes import GaussianNB

In [445]:
def make_data(db):
    X, Y = [], []
    range_dict = dict()
    start_idx = 0
    cnt = 0
    for tr_id, traj in db.iteritems():
        piece_match = match_res[tr_id]
        for old_idx, point in traj:
            obsv = point[2:-2]
            rid, lat, lng = piece_match[old_idx][3:6]
            x, y, _, _ = utm.from_latlon(lat, lng)
            cid = grid.utm2cell(x, y)
            state = -1
            for big_id, cells in slice_map[rid].iteritems():
                if cid in cells:
                    state = big_id
            if state == -1:
                cnt += 1
            X.append(obsv)
            Y.append(state)
        range_dict[tr_id] = (start_idx, len(X))
        start_idx = len(X)
    print cnt, len(X)
    return X, Y, range_dict
X_train, Y_train, _ = make_data(train)
X_test, Y_test, range_dict = make_data(new_test)

0 9371
0 3123


In [446]:
model = RandomForestClassifier()
model.fit(X_train, Y_train)

RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
            max_depth=None, max_features='auto', max_leaf_nodes=None,
            min_impurity_split=1e-07, min_samples_leaf=1,
            min_samples_split=2, min_weight_fraction_leaf=0.0,
            n_estimators=10, n_jobs=1, oob_score=False, random_state=None,
            verbose=0, warm_start=False)

In [447]:
model.classes_, len(model.classes_)

(array([  0,   1,   2,   3,   4,   9,  13,  25,  35,  37,  41,  56,  57,
         58,  84,  88,  92,  95, 114, 115, 116, 117, 118, 119, 120, 125,
        126, 127, 129, 130, 131, 133, 145, 146, 169, 170, 171, 172, 173,
        174, 175, 176, 177, 178, 181, 182, 183, 194, 195, 196, 201, 202,
        203, 204, 205, 206, 207, 210, 211, 213, 214, 215, 216, 217, 218,
        220, 224, 225, 226, 227, 228, 229, 231, 233, 234, 235, 236, 237,
        238, 239, 240, 241, 244, 247, 248, 249, 250, 251, 252, 255, 256,
        257, 258, 259, 260, 261, 262, 264, 265, 267, 268, 270, 272, 276,
        277, 278, 279, 280, 281, 283, 284, 285, 286, 287, 288, 289, 297,
        298, 299, 300, 306, 308, 320, 325, 340, 344, 346, 347]), 128)

In [448]:
Y_predict = model.predict_proba(X_test)

In [449]:
help_dict = dict()
min_len = 10
for tr_id in range_dict.iterkeys():
    s_idx, e_idx = range_dict[tr_id]
    help_dict[tr_id] = []
    for i, piece in enumerate(Y_predict[s_idx:e_idx]):
        length = len(set(piece)) - 1
#         k = min(min_len, length)
        k = length
        idxes = sorted(range(len(piece)), key=lambda x: piece[x], reverse=True)[:k]
        prob_info = dict()
        for idx in idxes:
            prob_info[model.classes_[idx]] = piece[idx]
        help_dict[tr_id].append(prob_info)

In [451]:
summary = dict()
cnt = 0
# statist = []
result = []
min_len = 10
for tr_id in range_dict.iterkeys():
    s_idx, e_idx = range_dict[tr_id]
    for i, piece in enumerate(Y_predict[s_idx:e_idx]):
        length = len(set(piece)) - 1
        k = min(min_len, length)
#         k = length
        result.append(length)
        idxes = sorted(range(len(piece)), key=lambda x: piece[x], reverse=True)[:k]
        rids = [model.classes_[idx] for idx in idxes]
        if Y_test[i + s_idx] in rids:
    #         statist.append((i, rids.index(Y_test[i]), length))
            cnt += 1
        else:
            print 'TrajID=', tr_id, 'i=',i
(len(Y_predict) - cnt, cnt * 1.0 / len(Y_test))

TrajID= 0 i= 2
TrajID= 1 i= 9
TrajID= 1 i= 16
TrajID= 1 i= 18
TrajID= 1 i= 19
TrajID= 7 i= 3
TrajID= 8 i= 22
TrajID= 8 i= 50
TrajID= 8 i= 51
TrajID= 8 i= 52
TrajID= 8 i= 53
TrajID= 8 i= 54
TrajID= 8 i= 66
TrajID= 8 i= 70
TrajID= 8 i= 71
TrajID= 8 i= 72
TrajID= 8 i= 75
TrajID= 8 i= 76
TrajID= 8 i= 84
TrajID= 9 i= 24
TrajID= 9 i= 25
TrajID= 9 i= 52
TrajID= 9 i= 81
TrajID= 10 i= 2
TrajID= 10 i= 3
TrajID= 10 i= 41
TrajID= 11 i= 49
TrajID= 12 i= 15
TrajID= 13 i= 81
TrajID= 13 i= 109
TrajID= 14 i= 20
TrajID= 14 i= 29
TrajID= 14 i= 30
TrajID= 14 i= 31
TrajID= 14 i= 185
TrajID= 18 i= 55
TrajID= 22 i= 35
TrajID= 23 i= 26
TrajID= 25 i= 0
TrajID= 29 i= 20
TrajID= 29 i= 21
TrajID= 29 i= 22
TrajID= 30 i= 12
TrajID= 35 i= 7
TrajID= 35 i= 8
TrajID= 35 i= 9
TrajID= 35 i= 16
TrajID= 38 i= 5
TrajID= 38 i= 6
TrajID= 38 i= 19
TrajID= 38 i= 21
TrajID= 39 i= 0
TrajID= 39 i= 1
TrajID= 39 i= 2
TrajID= 39 i= 23
TrajID= 40 i= 0
TrajID= 40 i= 1
TrajID= 41 i= 10
TrajID= 41 i= 21
TrajID= 43 i= 0
TrajID= 43 i= 1
Tr

(98, 0.9686199167467179)

In [175]:
summary

{1: (5, 0.9983085250338295),
 2: (5, 0.9983085250338295),
 3: (5, 0.9983085250338295),
 4: (5, 0.9983085250338295),
 5: (5, 0.9983085250338295),
 6: (5, 0.9983085250338295),
 7: (5, 0.9983085250338295),
 8: (5, 0.9983085250338295),
 9: (5, 0.9983085250338295),
 10: (5, 0.9983085250338295)}

In [405]:
np.mean(result), np.median(result), max(result), min(result)

(3.778613472858077, 3.0, 14, 1)

In [ ]:
plt.figure(figsize=(20, 10))
# plt.plot(one_ids.keys(), one_ids.values())
# plt.plot(two_ids.keys(), two_ids.values())
plt.plot(summary.keys(), summary.values())
xticks = (str(x) for x in summary.keys())
plt.xticks(summary.keys(), xticks)
# plt.legend(['ID1', 'ID1ID2', 'Context_ID1'], loc='best')
plt.title('RF predict_proba result')
plt.xlabel('min candidate')
plt.ylabel('predict_proba result')
plt.show()
# plt.savefig('./display/deepin/RF_predict_proba.png')

In [110]:
pd.DataFrame(statist, columns=['i', 'index', 'length']).describe()

,i,index,length
count,2775.000000,2775.000000,2775.000000
mean,1508.603964,0.405405,4.777658
std,876.029164,0.653568,3.555402
min,2.000000,0.000000,1.000000
25%,762.500000,0.000000,2.000000
50%,1489.000000,0.000000,4.000000
75%,2221.500000,1.000000,7.000000
max,3087.000000,2.000000,26.000000


# Layer 2

In [440]:
uni_hmm = reload(uni_hmm)

In [441]:
sag = uni_hmm.Sag(grid, slice_map, 1)

In [442]:
A, cell_db = sag.get_trans_mat(match_res, gps3.keys())

In [443]:
B, D = sag.get_obsv_mat(train, match_res, version=0)
uni_cells = sag.get_cells(D.keys())
len(uni_cells)

481

In [444]:
# clean test dataset
noise = reload(noise)
new_test = noise.clean_testset(sag, test, match_res, B, debug=True)

TrajID=9 Before=85 End=84
TrajID=10 Before=106 End=105
TrajID=11 Before=74 End=73
TrajID=18 Before=127 End=126
TrajID=29 Before=121 End=120


In [411]:
cell_pred = dict()
statistic = []
failed_trajs = []
for tr_id, traj in new_test.iteritems():
    speeds = [point[1][-2] for point in traj]
    k = math.floor(np.std(speeds))
    noexcept = False
    cnt = 0
    while noexcept is False and cnt < 10:
        noexcept = True
        try:
            sl, max_prob = sag.viterbi_(A, B, traj, k, help_dict[tr_id])
        except:
            noexcept = False
            k += 1
            cnt += 1
#     prob = sag.backforward(A, B, traj, match_res[tr_id], k, debug=False)
    prob = 0
    statistic.append((tr_id, max_prob, prob))
    cell_pred[tr_id] = sl
    print 'TrajID=' + str(tr_id) + 'Len=' + str(len(traj)), 'Out=' + str(len(sl))
print 'failed:', failed_trajs, len(failed_trajs)

TrajID=0Len=5 Out=5
TrajID=1Len=31 Out=31
TrajID=2Len=19 Out=19
TrajID=3Len=8 Out=8
TrajID=4Len=10 Out=10
TrajID=5Len=12 Out=10
TrajID=6Len=6 Out=6
TrajID=7Len=25 Out=25
TrajID=8Len=90 Out=90
TrajID=9Len=81 Out=90
TrajID=10Len=103 Out=103
TrajID=11Len=72 Out=103
TrajID=12Len=103 Out=103
TrajID=13Len=240 Out=240
TrajID=14Len=206 Out=206
TrajID=15Len=10 Out=10
TrajID=16Len=18 Out=18
TrajID=17Len=86 Out=86
TrajID=18Len=127 Out=127
TrajID=19Len=58 Out=58
TrajID=20Len=61 Out=58
TrajID=21Len=56 Out=56
TrajID=22Len=42 Out=42
TrajID=23Len=49 Out=49
TrajID=24Len=45 Out=45
TrajID=25Len=105 Out=105
TrajID=26Len=37 Out=37
TrajID=27Len=65 Out=65
TrajID=28Len=94 Out=94
TrajID=29Len=117 Out=117
TrajID=30Len=20 Out=20
TrajID=31Len=22 Out=22
TrajID=32Len=15 Out=15
TrajID=33Len=36 Out=36
TrajID=34Len=60 Out=60
TrajID=35Len=18 Out=18
TrajID=36Len=25 Out=25
TrajID=37Len=29 Out=29
TrajID=38Len=21 Out=21
TrajID=39Len=27 Out=27
TrajID=40Len=22 Out=22
TrajID=41Len=44 Out=44
TrajID=42Len=14 Out=14
TrajID=43Len

In [412]:
uni_prec, uni_prec_dict = sag.evaluate(new_test, cell_pred, match_res)

In [413]:
precs = [(tr_id, prec) for tr_id, prec in uni_prec_dict.iteritems()]
df_prob = pd.DataFrame(statistic, columns=['TrajID', 'MaxProb', 'RealProb'])
df_prec = pd.DataFrame(precs, columns=['TrajID', 'Precision'])

In [414]:
merged = pd.merge(df_prob, df_prec).sort_values(by=['Precision'], ascending=False).reset_index(drop=True)
error_ids = list(merged[merged['Precision'] > 100]['TrajID'].values)
merged

,TrajID,MaxProb,RealProb,Precision
0,9,8.588969,0,1022.008425
1,11,15.367890,0,781.048487
2,15,0.446123,0,622.107284
3,8,8.588969,0,142.623620
4,60,2.619483,0,116.911197
5,62,0.778901,0,105.447639
6,72,0.724445,0,95.247936
7,10,15.367890,0,88.904549
8,6,0.219895,0,85.245868
9,33,4.432622,0,79.589876


In [415]:
np.median(uni_prec), np.median(uni_prec_dict.values()), np.mean(uni_prec), len(uni_prec_dict)

(29.005371723225302, 38.49029887817497, 109.57029730252424, 69)

In [119]:
with open(data_path + 'result/' + 'ID1ID2_context.pkl', 'w') as fout:
    pickle.dump([uni_prec, uni_prec_dict], file=fout)

# Display

In [51]:
sum1, sum2 = dict(), dict()
r_rid = 0
for rid in D.iterkeys():
    sum1[r_rid] = len(D[rid])
    r_rid += 1
for obsv in B.iterkeys():
    num = len(B[obsv])
    if not sum2.has_key(num):
        sum2[num] = 0
    sum2[num] += 1

In [57]:
plot = reload(plot)

In [50]:
len(B)

437

In [58]:
plot.draw_barplot(sum2, 'n(obsv) of n(road)', 'n(road)', 'n(obsv)')